In [ ]:
In a jupyternotebook lets explore the following:

import sqlite3
import pandas as pd

# Connect to SQLite database
database_path="../Datasets/database/bank.db"
conn = sqlite3.connect(database_path)

# Create a cursor object
cursor = conn.cursor()


-- Demonstrates race conditions
-- Uses bank.db

-- Connection 1
SELECT "balance" FROM "accounts" WHERE "id" = 3;

-- Connection 2
SELECT "balance" FROM "accounts" WHERE "id" = 3;

-- Connection 1
UPDATE "accounts" SET "balance" = "balance" + 30 WHERE "id" = 1;

-- Connection 2
UPDATE "accounts" SET "balance" = "balance" + 30 WHERE "id" = 1;

-- Connection 3
SELECT "balance" FROM "accounts" WHERE "id" = 1;
UPDATE "accounts" SET "balance" = "balance" - 30 WHERE "id" = 1;

-- Connection 1
UPDATE "accounts" SET "balance" = "balance" - 30 WHERE "id" = 3;

-- Connection 2
UPDATE "accounts" SET "balance" = "balance" - 30 WHERE "id" = 3;


-- Demonstrates race conditions
-- Uses bank.db

-- Connection 1
SELECT "balance" FROM "accounts" WHERE "id" = 3;

-- Connection 2
SELECT "balance" FROM "accounts" WHERE "id" = 3;

-- Connection 1
UPDATE "accounts" SET "balance" = "balance" + 30 WHERE "id" = 1;

-- Connection 2
UPDATE "accounts" SET "balance" = "balance" + 30 WHERE "id" = 1;

-- Connection 3
SELECT "balance" FROM "accounts" WHERE "id" = 1;
UPDATE "accounts" SET "balance" = "balance" - 30 WHERE "id" = 1;

-- Connection 1
UPDATE "accounts" SET "balance" = "balance" - 30 WHERE "id" = 3;

-- Connection 2
UPDATE "accounts" SET "balance" = "balance" - 30 WHERE "id" = 3;

